# DL Assignment 2

Ishaan Reddy
SCDS
210200019

In [1]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import keras
tf.random.set_seed(42)

import numpy as np
np.random.seed(42)

import matplotlib.pyplot as plt
%matplotlib inline

import glob
import PIL
from PIL import Image

2.10.1


In [2]:
imgFiles = glob.glob(r'C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy/*/*.jpg')


for item in imgFiles:
    print(item)

C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (1).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (10).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (11).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (12).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (13).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (14).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (15).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (16).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning\Felis_Taxonomy\Felis_Taxonomy\african-wildcat\af (17).jpg
C:\Users\ishaa\OneDrive\Documents\Deep Learning

In [3]:
X = []
y = []

for fName in imgFiles:

  # Prepare the dataset and populate X and y
    X_i = Image.open(fName) 
    X_i = X_i.resize((299,299)) # To make them approriate to Xception model when using Transfer Learning
    X_i = np.array(X_i) / 255.0 # Normalize to range 0.0 to 1.0 (not stretching, only scaling)

    X.append(X_i)

    label = fName.split("\\")
    y_i = label[-2] 

    y.append(y_i)


In [4]:
from sklearn.preprocessing import LabelEncoder
lEncoder = LabelEncoder()
y = lEncoder.fit_transform(y)

print(set(y))
print(lEncoder.classes_)

X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)

{0, 1, 2, 3, 4, 5, 6}
['african-wildcat' 'blackfoot-cat' 'chinese-mountain-cat' 'domestic-cat'
 'european-wildcat' 'jungle-cat' 'sand-cat']
(519, 299, 299, 3)
(519,)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
                                                    stratify=y, random_state=42)

print("X_train_shape: {}".format(X_train.shape))
print("X_test_shape: {}".format(X_test.shape))

X_train_shape: (389, 299, 299, 3)
X_test_shape: (130, 299, 299, 3)


In [6]:
mu = X_train.mean()
std = X_train.std()

X_train_std = (X_train-mu)/std
X_test_std = (X_test-mu)/std

a. Total number of images in the dataset = 519

b. Number of images present per class
    Class: african-wildcat, Count: 91
    Class: blackfoot-cat, Count: 79
    Class: chinese-mountain-cat, Count: 42
    Class: domestic-cat, Count: 64
    Class: european-wildcat, Count: 85
    Class: jungle-cat, Count: 86
    Class: sand-cat, Count: 72 
    
c. Total number of images used for training and testing.
    Train = 389
    Test = 130

## Transfer Learning

### Model 2: DenseNet121

In [7]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, GlobalAveragePooling2D


base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(299, 299, 3))


for layer in base_model.layers:
    layer.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)
predictions = Dense(7, activation='softmax')(x) 


model_TL2 = Model(inputs=[base_model.input], outputs=[predictions])

In [8]:
model_TL2.compile(loss='sparse_categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

callbacks_TL2 = [
             keras.callbacks.ModelCheckpoint("bestTL2.h5",
                                             monitor='val_accuracy',
                                             save_weights_only=True,
                                             save_best_only=True)]

history_TL2 = model_TL2.fit(x = X_train_std, y = y_train, epochs=10,
                          validation_split=0.1, batch_size=16, callbacks=callbacks_TL2)

Epoch 1/10
22/22 [==============================] - 155s 5s/step - loss: 2.0723 - accuracy: 0.2486 - val_loss: 1.6754 - val_accuracy: 0.4359
Epoch 2/10
22/22 [==============================] - 90s 4s/step - loss: 1.1688 - accuracy: 0.5943 - val_loss: 1.3130 - val_accuracy: 0.5641
Epoch 3/10
22/22 [==============================] - 97s 4s/step - loss: 0.8886 - accuracy: 0.6886 - val_loss: 1.1039 - val_accuracy: 0.6410
Epoch 4/10
22/22 [==============================] - 108s 5s/step - loss: 0.6142 - accuracy: 0.8086 - val_loss: 0.9490 - val_accuracy: 0.6154
Epoch 5/10
22/22 [==============================] - 93s 4s/step - loss: 0.5500 - accuracy: 0.8371 - val_loss: 0.8583 - val_accuracy: 0.6154
Epoch 6/10
22/22 [==============================] - 49s 2s/step - loss: 0.4589 - accuracy: 0.8600 - val_loss: 0.7813 - val_accuracy: 0.6667
Epoch 7/10
22/22 [==============================] - 54s 2s/step - loss: 0.3895 - accuracy: 0.8857 - val_loss: 0.7033 - val_accuracy: 0.6923
Epoch 8/10
22/22 [

### Evaluate the model 

In [9]:
testLoss_TL, testAccuracy_TL = model_TL2.evaluate(x = X_test_std, y = y_test)
print("Test-loss: %f, Test-accuracy: %f" % (testLoss_TL, testAccuracy_TL))

5/5 [==============================] - 23s 4s/step - loss: 0.6035 - accuracy: 0.8308
Test-loss: 0.603462, Test-accuracy: 0.830769


### Update with best weights 

In [10]:
model_TL2.load_weights("bestTL2.h5")

testLoss_TL, testAccuracy_TL = model_TL2.evaluate(x = X_test_std, y = y_test)

print("Test-loss: %f, Test-accuracy: %f" % (testLoss_TL, testAccuracy_TL))

5/5 [==============================] - 22s 4s/step - loss: 0.6635 - accuracy: 0.8077
Test-loss: 0.663470, Test-accuracy: 0.807692


In [11]:
model_TL2.save('/content/drive/MyDrive/DL_Models/02_Xception_TransferLearning_Best_Model.h5')

In [12]:
from numpy import save

save('/content/drive/MyDrive/DL_Models/X_train_std.npy', X_train_std)
save('/content/drive/MyDrive/DL_Models/X_test_std.npy', X_test_std)

save('/content/drive/MyDrive/DL_Models/y_train.npy', y_train)
save('/content/drive/MyDrive/DL_Models/y_test.npy', y_test)